In [1]:
print("hello")

hello


In [ ]:
import psycopg2
import sys
import os
from psycopg2.extras import RealDictCursor
import csv
from datetime import datetime
import logging
import argparse
import json
import snowflake.connector

def run_snwflk_query(table_schema,table_name,snwflk_query):
    '''This Function execute Snowflake Query command'''
    try:
        
        snow_con = snowflake.connector.connect(
        user='SNOWPIPE_PROD',
        account='wq16105.eu-west-1',
        #authenticator='externalbrowser',
        password='123',
        WAREHOUSE = 'FLK_WH',
        DATABASE = 'FLK_DB'
        )
        rl_cmnd=f"USE ROLE FLK_DUB_ROL_DATALAKEDEPLOY_PRD"
        print(f"Role Command:{rl_cmnd}")
        schema_cmnd=f"USE SCHEMA FLK_DB.{table_schema}"
        print(f"Schema Command:{schema_cmnd}")
        wr_cmnd=f"USE WAREHOUSE FLK_WH"
        print(f"Warehouse Command:{wr_cmnd}")
        with snow_con:
             print(" ============= Inside Snow CONNECTION ===================")
             cur = snow_con.cursor()
             cur.execute(rl_cmnd)
             cur.execute(schema_cmnd)
             cur.execute(wr_cmnd)
             
             cur.execute(snwflk_query)
             one_row = cur.fetchone()
             print(f"Snowflake Query Return:{one_row[0]}")
             snow_queryid = cur.sfqid
             print(f"Snowflake Query Return:{snow_queryid}")
             status = snow_con.get_query_status_throw_if_error(snow_queryid)
             finalstatus=f"{status} {one_row[0]}"
             
        return finalstatus,one_row[0]
        print(" ============= Snow query execute successfully ===================")
    except Exception as err:
        err =str(err).replace("\n","").replace("'","''")

def audit_write(file_name,table_schema,table_name,rs_cnt,sflk_cnt,status):
    '''This Function write audit information after each unload and copy operation with status'''
    with open(file_name, mode='a',newline='') as table_unload_copy:
        table_unload_copy_w = csv.writer(table_unload_copy, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        table_unload_copy_w.writerow([table_schema,table_name,rs_cnt,sflk_cnt,status])

def run_redshift_query(con,table_schema,table_name,cnt_query):
    '''This Function perform unload operation from redshift'''
    try:
        with con:
            print(" ============= Inside Redshift CONNECTION ===================")
            cur = con.cursor()
            cur.execute(cnt_query)
            rows = cur.fetchall()
        print(" ============= Redshift query execute successfully ===================")
        print(rows[0]['count'])
        return 'Success',rows[0]['count']
    except psycopg2.DatabaseError as err:
        err =str(err).replace("\n","").replace("'","''")
        return(err)

def main():	
           con = psycopg2.connect(dbname= 'cuscdpdb', host='cdp-redshift-prod.ckpf5iovcylg.eu-west-1.redshift.amazonaws.com', 
                         port= 5439, user= 'cdp_readonly', password='CDPProd@Mat123' ,cursor_factory=RealDictCursor)
           print("######## Create Redshift Connection #######")
           
           audit_write('etl0_miss_21_02_22.csv','table_schema','table_name','Redshift_Count','Snowflake Count','status')
           with open("file_list_etl0.csv") as f:
               #print(f.read())
               line_list = f.read().splitlines()
               for each_line in line_list:
                   each_line_list=each_line.split(',')
                   table_schema=each_line_list[0]
                   table_name=each_line_list[1]
                   print(f"schema name : {table_schema}")
                   print(f"table name : {table_name}")
                   cnt_query=f"select count(1) from {table_schema}.{table_name} where deleteflag=false"
                   rs_status,rs_cnt=run_redshift_query(con,table_schema,table_name,cnt_query)
                   sflk_status,sflk_cnt=run_snwflk_query(table_schema,table_name,cnt_query)
                   if rs_cnt != sflk_cnt:
                       status='Not_Matched'
                   else:
                       status='Matched'
                   
                   
                   audit_write('etl0_miss_21_02_22.csv',table_schema,table_name,rs_cnt,sflk_cnt,status)    

if __name__=="__main__":
    try:
        main()
    except Exception as e:
        logging.exception(f"Exception occurred for error : {e}")
        sys.exit(1)                 
                   